# Calculate satelliete position from broadcast orbital parameters

Beacause the pertutbed motion, the calculation of the satellite position is more complex than the Keplerian model.

The orbit of an satellite (and the clock of the sattelite) can be determined by more parameters.

|name|notation|dimension|var name|position in the RINEX nav file|
|-----|-----|-----|-----|-----|
|Reference epoch| $t_{0e}$ |time|t0e|row 4, 1st field|
|Mean anomaly in the reference epoch| $M_{0}$ |angle|m0|row 2, 4th field|
|Square root of semimajor axis of orbital ellipse| $\sqrt{a}$ |distance (sqrt)|sqrta|row 3, 4th field|
|Numerical eccentricity of ellipse| $e$ |-|ecc|row 3, 2nd field|
|Mean motion difference| $\Delta n$ |angle/time|delta_n|row 2, 3rd field|
|Longitude of ascending node in the reference epoch| $\Omega_{0}$ |angle|uom0|row 4, 3rd field|
|Rate of nodes right ascension| $\dot{\varOmega}$ |ange/time|dot_uom|row 5, 4th field|
|Inclination of orbital plane in the reference eopch| $i_{0}$ |angle|i0|row 5, 1st field|
|Rate of inclination angle| $\dot{i}$ |angle/time|dot_i|row 6, 1st field|
|Argument of perigee| $\omega$ |angle|lom|row 5, 3rd field|
|Cosine part of latitude argument correction| $c_{uc}$ |angle|cuc|row 3, 1st field|
|Sine part of latitude argument correction| $c_{us}$ |angle|cus|row 3, 3rd field|
|Cosine part of orbital radius correction| $c_{rc}$ |distance|crc|row 5, 2nd field|
|Sine part of orbital radius correction| $c_{rs}$ |distance|crs|row 2, 2nd field|
|Cosine part of inclination correction| $c_{ic}$ |angle|cic|row 4, 2nd field|
|Sine part of inclination correction| $c_{is}$ |angle|cis|row 4, 4th field|
|Clock offset| $a_{0}$ |time|clk_a0|first row, the third field from the back|
|Clock drift| $a_{1}$ |-|clk_a1|first row, the second field from the back|
|Clock drift rate| $a_{2}$ |1/time|clk_a2|first row, last field|

The value of $\mu_{Earth}=3.986005\cdot10^{14}\frac{m^{3}}{s^{2}}$

The value of $\omega_{Earth}=0.00007292115147$

The value of $\pi=3.1415926535898$

In [54]:
#Import required modules
from math import sqrt, sin, cos, tan, atan, atan2
import numpy as np

#constants
mue=398600500000000.0
lome=0.00007292115147
pi=3.1415926535898

#data from RINEX navigation file
navrintxt="""31 18  9  5  7 59 44.0 0.957404263318D-04-0.261479726760D-11 0.000000000000D+00
    0.600000000000D+01 0.258750000000D+02 0.453197435135D-08-0.284228546039D+01
    0.142678618431D-05 0.884578982368D-02 0.105015933514D-04 0.515373404312D+04
    0.287984000000D+06-0.154599547386D-06-0.133607394295D+01-0.353902578354D-07
    0.961978018040D+00 0.178906250000D+03-0.114071783319D+00-0.779175302057D-08
   -0.412874345823D-09 0.100000000000D+01 0.201700000000D+04 0.000000000000D+00
    0.000000000000D+00 0.000000000000D+00-0.135041773319D-07 0.600000000000D+01
    0.285576000000D+06 0.000000000000D+00 0.000000000000D+00 0.000000000000D+00"""

#Split to rows the RINEX data. Put an empty string to the firs for the 0 index
rnrw=['']+navrintxt.split('\n')

#Define a fuction for read a field from RINEX data
def rndat(rnrow, idx):
    return float(rnrow[3+(idx-1)*19:18+(idx-1)*19])*10**int(rnrow[19+(idx-1)*19:22+(idx-1)*19])

#Read data from the RINEX text
t0e=rndat(rnrw[4],1)
m0=rndat(rnrw[2],4)
sqrta=rndat(rnrw[3],4)
a=sqrta**2
ecc=rndat(rnrw[3],2)
delta_n=rndat(rnrw[2],3)
uom0=rndat(rnrw[4],3)
dot_uom=rndat(rnrw[5],4)
i0=rndat(rnrw[5],1)
dot_i=rndat(rnrw[6],1)
lom=rndat(rnrw[5],3)
cuc=rndat(rnrw[3],1)
cus=rndat(rnrw[3],3)
crc=rndat(rnrw[5],2)
crs=rndat(rnrw[2],2)
cic=rndat(rnrw[4],2)
cis=rndat(rnrw[4],4)
clk_a0=rndat(rnrw[1],2)
clk_a1=rndat(rnrw[1],3)
clk_a2=rndat(rnrw[1],4)

t=t0e+1600

t0e, m0*(180/pi), sqrta, a, ecc, delta_n, uom0, dot_uom, i0*(180/pi), dot_i, lom, cuc, cus, crc, crs, cic, cis, clk_a0, clk_a1, clk_a2

(287984.0,
 -162.85096105174475,
 5153.73404312,
 26560974.587214023,
 0.00884578982368,
 4.53197435135e-09,
 -1.33607394295,
 -7.79175302057e-09,
 55.11728041805165,
 -4.12874345823e-10,
 -0.114071783319,
 1.42678618431e-06,
 1.05015933514e-05,
 178.90625,
 25.874999999999996,
 -1.54599547386e-07,
 -3.53902578354e-08,
 9.574042633180001e-05,
 -2.6147972675999996e-12,
 0.0)

Calculation of the position of the satellite in the $t$ epoch.

Time from the reference epoch:

$$t_{k}=t-t_{0e}$$

The mean anomaly:

$$M_{k}=M_{0}+\left(\sqrt{\frac{\mu_{Earth}}{a^{3}}}+\Delta n\right)t_{k}$$

In [55]:
tk=t-t0e
mak=m0+(sqrt(mue/a**3)+delta_n)*tk
tk, mak, mak*(180/pi)

(1600.0, -2.608920102436533, -149.4801109564514)

The eccentric anomaly is similar than the Keplerian case.
The calculation is an iterative method, initially $E_{k}=M_{k}$.

$$E_{k}=M_{k}+e\sin E_{k}$$

In [56]:
#the initial value is the mean anomaly
eak=mak
#the iteration loop
while True:
    old_eak=eak
    eak=mak+ecc*sin(eak)
    if abs(eak-old_eak)<0.0000000000001:
        break
eak, eak*(180/pi)

(-2.6133783085062454, -149.73554734844555)

The true anomaly is also simlar than the Keplerian case:

$$\upsilon_{k}=\arctan\left(\frac{\sqrt{1-e^{2}}\sin E_{k}}{\cos E_{k}-e}\right)$$

In [57]:
tak=atan2(sqrt(1-ecc**2)*sin(eak), cos(eak)-ecc)

#another calulation for check
tak2=2*atan2(sqrt((1+ecc)/(1-ecc))*sin(eak/2),cos(eak/2))

tak*(180/pi), tak2*(180/pi)

(-149.9900162042633, -149.99001620426327)

Argument of latitude:

$$u_{k}=\omega+\upsilon_{k}+c_{uc}\cos2\left(\omega+\upsilon_{k}\right)+c_{us}\sin2\left(\omega+\upsilon_{k}\right)$$

Radial distance:

$$r_{k}=a\left(1-e\cos E_{k}\right)+c_{rc}\cos2\left(\omega+\upsilon_{k}\right)+c_{rs}\sin2\left(\omega+\upsilon_{k}\right)$$

Inclination of orbital plane:

$$i_{k}=i_{0}+\dot{i}t_{k}+c_{ic}\cos2\left(\omega+\upsilon_{k}\right)+c_{is}\sin2\left(\omega+\upsilon_{k}\right)$$

Longitude of ascending node:

$$\lambda_{k}=\varOmega_{0}+\left(\dot{\varOmega}-\omega_{E}\right)t_{k}-\omega_{E}t_{0e}$$

In [58]:
aolk=lom+tak+cuc*cos(2*(lom+tak))+cus*sin(2*(lom+tak))
rk=a*(1-ecc*cos(eak))+crc*cos(2*(lom+tak))+crs*sin(2*(lom+tak))
ik=i0+dot_i*tk+cic*cos(2*(lom+tak))+cis*sin(2*(lom+tak))

loank=(uom0+(dot_uom-lome)*tk-lome*t0e)%(2*pi)

aolk,rk,ik,loank*(180/pi)

(-2.7318827632821376,
 26764046.33329098,
 0.9619772260414272,
 153.54444375540623)

The ECEF coordinates of the satellite in t time:

$$\left[\begin{array}{c}
X\\
Y\\
Z
\end{array}\right]=R_{Z}\left(-\lambda_{k}\right)\cdot R_{X}\left(-i_{k}\right)\cdot R_{Z}\left(-u_{k}\right)\cdot\left[\begin{array}{c}
r_{k}\\
0\\
0
\end{array}\right]$$

In [59]:
rm_loank=np.array([[ cos(loank), -sin(loank), 0.0],
                   [ sin(loank),  cos(loank), 0.0],
                   [        0.0,         0.0, 1.0]])

rm_ik=np.array([[1.0,      0.0,      0.0],
                [0.0,  cos(ik), -sin(ik)],
                [0.0,  sin(ik),  cos(ik)]])

rm_aolk=np.array([[ cos(aolk), -sin(aolk), 0.0],
                  [ sin(aolk),  cos(aolk), 0.0],
                  [       0.0,        0.0, 1.0]])

coordk=rm_loank @ rm_ik @ rm_aolk @ np.array([[rk],[0],[0]])

coordk

array([[ 24694509.07145114],
       [ -5477966.23643446],
       [ -8745700.87317465]])

The offset of the clock in t time:

$$a_{k}=a_{0}+a_{1}t_{k}+a_{2}t_{k}^{2}$$

In [60]:
clk_ak=clk_a0+clk_a1*tk+clk_a2*tk**2

clk_ak

9.573624265617186e-05

## References

Bernhard Hofmann-Wellenhof, Herbert Lichtenegger, Elmar Wasle: GNSS — Global Navigation Satellite Systems, Springer-Verlag Wien, 2008. Subsection 3.4.2

https://gssc.esa.int/navipedia/index.php/GPS_and_Galileo_Satellite_Coordinates_Computation